In [51]:
import pandas as pd
import numpy as np
import requests
import time
from tqdm import tqdm_notebook

### 국토교통부 실거래가 공개시스템 활용법
* http://rtdown.molit.go.kr/
* 주소 형식은 우선 지번주소로 진행, 도로명주소로 했을 경우 아래 parameter값 설정 시 type값 맞춰서 변경
* 조건 설정 후 다운로드 받을 때, EXCEL 형식으로 선택(CSV) 선택 시 알수없는 인코딩 형식으로 다운로드되어 읽지 못함)
* 엑셀로 다운로드 파일 열어서 설명 부분 지우고, 칼럼명과 데이터값만 남기고 CSV로 저장하면 pandas dataframe으로 읽기 가능
* 거래금액 칼럼에 천단위 컴마(,)까지 저장되지 않도록, 엑셀에서 셀 서식에서 수정 후 저장

In [52]:
data = pd.read_csv("../data_apt/seoul_20190611161639.csv")

In [53]:
data.shape

(1458, 12)

In [54]:
data.index

RangeIndex(start=0, stop=1458, step=1)

In [55]:
data.head()

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201905,4,124000,4,1987,언주로
1,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,35.64,201905,4,135000,5,1982,개포로
2,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,42.55,201905,6,150000,1,1982,개포로
3,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,50.38,201905,6,199000,5,1982,개포로
4,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,58.08,201905,6,192000,3,1982,개포로


### vworld 오픈 API 활용법
* http://www.vworld.kr/
* 회원가입 후 "인증키 > 인증키 발급" 메뉴에서 인증키 발급(단, 4월 19일 이후 당분간 인증키 발급 중단 ㅠ.ㅠ)
* 발급되면 "인증키 > 인증키 관리" 메뉴에서 확인하여 사용 가능
* 일일 30,000건 사용 가능하므로 이보다 많을 때에는 날짜를 나누어 사용 필요
* Geocoder API 2.0 레퍼런스 참조하여 활용
* http://www.vworld.kr/dev/v4dv_geocoderguide2_s001.do
* 주소를 좌표(위/경도), 좌표를 주소로 변환할 수 있음

In [56]:
# 주소 > 좌표 변화을 위한 URL 호출 방식
# http://api.vworld.kr/req/address?service=address&request=getCoord&key=인증키&[요청파라미터]
# URL 조합을 위해 아래 2개의 변수에 URL과 API 인증키 저장
url = "http://api.vworld.kr/req/address?service=address&request=getCoord&key="
api_key = "" # API 인증키 입력

In [58]:
# data row 개수 확인
len(data)

1458

In [23]:
# for문 사용을 위해 range 확인
# range([start,] stop [,step]) 형식으로 활용
# stop 값만 넣으면, 0~stop값까지 1씩 카운트하며 리턴(ex : range(5) > 0, 1, 2, 3, 4, 5)
range(len(data))

range(0, 1458)

In [59]:
# for, range 구문 셈플 코드
for i in range(5):
    print(i)

0
1
2
3
4


In [60]:
# 첫번째(0번째) 열의 시구군 행 값 확인
data.loc[0, "시군구"]

'서울특별시 강남구 개포동'

In [61]:
# 시구군 + 공백 + 번지로 주소(address) 조합
address = data.loc[0, "시군구"] + " " + data.loc[0, "번지"]
print(address)

서울특별시 강남구 개포동 658-1


In [62]:
# 앞서 API 호출을 위해 조합한 url, api_key값과 주소를 결합하여 요청할 URL 생성
# type 파라미터(prarmeter)는 PARCEL : 지번주소, ROAD : 도로명주소
# 다운받을 때
request_url = url + api_key + "&type=PARCEL&address=" + address
print(request_url)

http://api.vworld.kr/req/address?service=address&request=getCoord&key=78A60B42-1A46-3D9B-AF34-DA3EB57C121D&type=PARCEL&address=서울특별시 강남구 개포동 658-1


In [63]:
# 조합한 URL 호출하여 리턴값 변수에 저장
# response값이 보통 200대면 정상, 300번대는 리다이렉션, 
# 400번대 클라이언트 에러, 500번대 서버 에러
r = requests.get(request_url)
r

<Response [200]>

In [64]:
# 리턴받은 json을 파싱하기 위해 json() 모듈 활용
geocoord_json = r.json()
geocoord_json

{'response': {'service': {'name': 'address',
   'version': '2.0',
   'operation': 'getCoord',
   'time': '740(ms)'},
  'status': 'OK',
  'input': {'type': 'PARCEL', 'address': '서울특별시 강남구 개포동 658-1'},
  'refined': {'text': '서울특별시 강남구 개포동 658-1',
   'structure': {'level0': '',
    'level1': '',
    'level2': '',
    'level3': '',
    'level4L': '',
    'level4LC': '',
    'level4A': '',
    'level4AC': '',
    'level5': '',
    'detail': ''}},
  'result': {'crs': 'EPSG:4326',
   'point': {'x': '127.057216059296', 'y': '37.4762144592679'}}}}

In [65]:
# csr은 응답결과 좌표계를 의미 (EPSG:4326 = WGS84 경위도)
# x좌표 = 경도, y좌표 = 위도
geo_x = float(geocoord_json['response']['result']['point']['x'])
geo_y = float(geocoord_json['response']['result']['point']['y'])
print(geo_x, geo_y)

127.057216059296 37.4762144592679


In [66]:
# time.sleep 이용하면, 딜레이타임 설정 가능
# 없어도 상관 없으나 서버/클라이언트간 속도 고려 필요 시 적용
for i in range(5):
    print(i)
    time.sleep(2)

0
1
2
3
4


In [67]:
# tqdm_notebook 활용하면, jupyter notebook에서 시각적으로 % 확인 가능
for i in tqdm_notebook(range(5)):
    print(i)
    time.sleep(2)

0
1
2
3
4



In [68]:
# for 문으로 순차적으로 경/위도 좌표값을 받아와 새로운 칼럼에 추가하는 코드 작성
# 우선 5개 정도 받아오도록 하고, 이후 이상 없으면 전체 적용
for i in tqdm_notebook(range(5)):
    address = data.loc[i, "시군구"] + " " + data.loc[i, "번지"]
    request_url = url + api_key + "&type=PARCEL&address=" + address
    r = requests.get(request_url)
    geocoord_json = r.json()
    data.loc[i, "geo_x"] = float(geocoord_json['response']['result']['point']['x'])
    data.loc[i, "geo_y"] = float(geocoord_json['response']['result']['point']['y'])

In [69]:
data.head(10)

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,geo_x,geo_y
0,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201905,4,124000,4,1987,언주로,127.057216,37.476214
1,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,35.64,201905,4,135000,5,1982,개포로,127.061647,37.478510
2,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,42.55,201905,6,150000,1,1982,개포로,127.061647,37.478510
3,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,50.38,201905,6,199000,5,1982,개포로,127.061647,37.478510
4,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,58.08,201905,6,192000,3,1982,개포로,127.061647,37.478510
5,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,41.98,201905,6,151500,5,1982,개포로,NaN,NaN
6,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,35.87,201905,6,137500,3,1982,개포로,NaN,NaN
7,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,41.98,201905,7,151000,5,1982,개포로,NaN,NaN
8,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,58.08,201905,7,193000,3,1982,개포로,NaN,NaN
9,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,56.57,201905,8,245000,5,1982,선릉로,NaN,NaN


In [70]:
del data["geo_x"]
del data["geo_y"]

In [71]:
data.head(7)

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201905,4,124000,4,1987,언주로
1,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,35.64,201905,4,135000,5,1982,개포로
2,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,42.55,201905,6,150000,1,1982,개포로
3,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,50.38,201905,6,199000,5,1982,개포로
4,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,58.08,201905,6,192000,3,1982,개포로
5,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,41.98,201905,6,151500,5,1982,개포로
6,서울특별시 강남구 개포동,141,141,0,개포주공 1단지,35.87,201905,6,137500,3,1982,개포로


In [ ]:
# 전체 주소 > 좌표값 받아오도록 수정
# for i in tqdm_notebook(range(5)):
for i in tqdm_notebook(len(data)):
    address = data.loc[i, "시군구"] + " " + data.loc[i, "번지"]
    request_url = url + api_key + "&type=PARCEL&address=" + address
    r = requests.get(request_url)
    geocoord_json = r.json()
    data.loc[i, "geo_x"] = float(geocoord_json['response']['result']['point']['x'])
    data.loc[i, "geo_y"] = float(geocoord_json['response']['result']['point']['y'])